In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load data
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [ ]:
# Create targets
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [ ]:
data_preprocessed['Excessive Absenteeism'] = targets

In [ ]:
data_preprocessed.head()

In [ ]:
# Checking for balance in dataset 
targets.sum() / targets.shape[0]

In [ ]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'],axis=1)
data_with_targets.head()

In [ ]:
# Select inputs for regression
unscaled_inputs = data_with_targets.iloc[:,:-1]
unscaled_inputs

In [ ]:
# Standardize data
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler()
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None):
        
        init_col_order = X.columns
        
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [ ]:
unscaled_inputs.columns.values

In [ ]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [ ]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [ ]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [ ]:
absenteeism_scaler.fit(unscaled_inputs)

In [ ]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

In [ ]:
scaled_inputs.shape

In [ ]:
# Split data into train and test, then shuffle
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

In [ ]:
# Check size of train data
print(x_train.shape, y_train.shape)

In [ ]:
# Check size of test data
print(x_test.shape, y_test.shape)

In [ ]:
# Training the model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

In [ ]:
# Train accuracy of model
reg.score(x_train, y_train)

In [ ]:
# Find intercept of model
reg.intercept_

In [ ]:
# Find coefficients of model
reg.coef_

In [ ]:
# Display coefficients in table
feature_name = unscaled_inputs.columns.values

In [ ]:
summary_table = pd.DataFrame (columns=['Feature name'], data=feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

In [ ]:
# Add intercept to table
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

In [ ]:
# Interpretting coefficients
summary_table['Odds ratio'] = np.exp(summary_table.Coefficient)

In [ ]:
summary_table

In [ ]:
summary_table.sort_values('Odds ratio', ascending=False)